# EXAMEN PARCIAL

1. (7 puntos) Implementar el algoritmo Erdős–Rényi y Barabasi-Albert para generar una red libre de escala, para su implementación no es permitido utilizar librerías de redes, su implementación debe recibir los parámetros necesarios.

In [1]:
from collections import defaultdict
import random

class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(set)
    
    def add_node(self, node):
        self.nodes.add(node)
    
    def add_edge(self, u, v):
        self.edges[u].add(v)
        self.edges[v].add(u)
    
    def get_degree(self, node):
        return len(self.edges[node])

In [2]:
def erdos_renyi(n, p):
    graph = Graph()
    
    # Añadir nodos
    for i in range(n):
        graph.add_node(i)
    
    # Añadir aristas
    for i in range(n):
        for j in range(i+1, n):
            if random.random() < p:
                graph.add_edge(i, j)
    
    return graph

In [4]:
def barabasi_albert(n, m):
    graph = Graph()
    
    # Inicializar con m nodos
    for i in range(m):
        graph.add_node(i)
    
    # Conectar los primeros m nodos entre sí
    for i in range(m):
        for j in range(i+1, m):
            graph.add_edge(i, j)
    
    # Añadir el resto de nodos
    for i in range(m, n):
        graph.add_node(i)
        
        # Seleccionar m nodos existentes para conectar
        existing_nodes = list(graph.nodes)
        probabilities = [graph.get_degree(node) / sum(graph.get_degree(n) for n in existing_nodes) for node in existing_nodes]
        
        selected_nodes = random.choices(existing_nodes, weights=probabilities, k=m)
        
        for node in selected_nodes:
            graph.add_edge(i, node)
    
    return graph

In [5]:
if __name__ == "__main__":

    er_graph = erdos_renyi(100, 0.1)
    print(f"Red aleatoria (Erdős–Rényi): {len(er_graph.nodes)} nodos, {sum(len(edges) for edges in er_graph.edges.values()) // 2} aristas")

    # Generar una red libre de escala usando Barabási-Albert
    ba_graph = barabasi_albert(100, 2)
    print(f"Red libre de escala (Barabási-Albert): {len(ba_graph.nodes)} nodos, {sum(len(edges) for edges in ba_graph.edges.values()) // 2} aristas")

Red aleatoria (Erdős–Rényi): 100 nodos, 515 aristas
Red libre de escala (Barabási-Albert): 100 nodos, 189 aristas


2. (5 puntos) Crear redes con 500 nodos y un m=3, utilizando los algoritmos de la pregunta 1 y utilizando la librería NetworkX, comparar las métricas de longitud de camino medio y coeficiente de agrupamiento, mostrar también la gráfica de distribución de grado (Explicar los resultados).

In [15]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter

# Función para calcular métricas
def calcular_metricas(G):
    try:
        avg_shortest_path = nx.average_shortest_path_length(G)
    except nx.NetworkXError:
        avg_shortest_path = float('inf')
    
    avg_clustering = nx.average_clustering(G)
    return avg_shortest_path, avg_clustering

# Función para graficar distribución de grados
def plot_degree_distribution(G, title):
    degrees = [G.degree(n) for n in G.nodes()]
    degree_count = Counter(degrees)
    x, y = zip(*degree_count.items())
    
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, alpha=0.7)
    plt.xscale('log')
    plt.yscale('log')
    plt.title(f'Distribución de grado - {title}')
    plt.xlabel('Grado (k)')
    plt.ylabel('Número de nodos con grado k')
    plt.savefig(f'degree_distribution_{title.lower().replace(" ", "_")}.png')
    plt.close()

# Parámetros
N = 500
M = 3
P = 2 * M / (N - 1)

# Generar redes con los algoritmos implementados
er_graph = erdos_renyi(N, P)
ba_graph = barabasi_albert(N, M)

# Convertir en formato NetworkX
er_nx = nx.Graph(er_graph)
ba_nx = nx.Graph(ba_graph)

# Generar redes con NetworkX
er_nx_lib = nx.erdos_renyi_graph(N, P)
ba_nx_lib = nx.barabasi_albert_graph(N, M)

# Calcular métricas
metricas = {
    "ER (propio)": calcular_metricas(er_nx),
    "BA (propio)": calcular_metricas(ba_nx),
    "ER (NetworkX)": calcular_metricas(er_nx_lib),
    "BA (NetworkX)": calcular_metricas(ba_nx_lib)
}

# Imprimir resultados
print("Modelo\t\t\tLong. camino medio\tCoef. agrupamiento")
for modelo, (apl, cc) in metricas.items():
    print(f"{modelo}\t{apl:.4f}\t\t\t{cc:.4f}")

# Generar gráficas de distribución de grado
plot_degree_distribution(er_nx, "ER (propio)")
plot_degree_distribution(ba_nx, "BA (propio)")
plot_degree_distribution(er_nx_lib, "ER (NetworkX)")
plot_degree_distribution(ba_nx_lib, "BA (NetworkX)")


KeyboardInterrupt: 

(8 puntos) Utilizando la librería Networkx analizar la red del dataset, calcular las métricas
vistas en clase, en base al análisis verificar si la red es aleatoria, mundo pequeño y/o libre de escala (Justifique su respuesta).

In [11]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Cargar los datos y crear el grafo
G = nx.Graph()
with open('ia-enron-only.csv', 'r') as f:
    for line in f:
        source, target = map(int, line.strip().split(','))
        G.add_edge(source, target)

# Calcular métricas
n = G.number_of_nodes()
m = G.number_of_edges()
avg_degree = 2 * m / n
avg_clustering = nx.average_clustering(G)
avg_shortest_path = nx.average_shortest_path_length(G)

# Calcular la distribución de grado
degree_sequence = [d for n, d in G.degree()]
degree_counts = Counter(degree_sequence)
x = list(degree_counts.keys())
y = list(degree_counts.values())

# Imprimir resultados
print(f"Número de nodos: {n}")
print(f"Número de aristas: {m}")
print(f"Grado promedio: {avg_degree:.2f}")
print(f"Coeficiente de clustering promedio: {avg_clustering:.4f}")
print(f"Longitud de camino más corto promedio: {avg_shortest_path:.4f}")

# Graficar la distribución de grado
plt.figure(figsize=(10, 6))
plt.scatter(x, y)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Grado (k)')
plt.ylabel('Número de nodos con grado k')
plt.title('Distribución de grado (escala log-log)')
plt.savefig('enron_degree_distribution.png')
plt.close()

# Comparar con una red aleatoria equivalente
G_random = nx.erdos_renyi_graph(n, p=(2*m)/(n*(n-1)))
random_avg_clustering = nx.average_clustering(G_random)
random_avg_shortest_path = nx.average_shortest_path_length(G_random)

print("\nComparación con red aleatoria equivalente:")
print(f"Coeficiente de clustering (aleatorio): {random_avg_clustering:.4f}")
print(f"Longitud de camino más corto (aleatorio): {random_avg_shortest_path:.4f}")

# Calcular el coeficiente de asortatividad
assortativity = nx.degree_assortativity_coefficient(G)
print(f"\nCoeficiente de asortatividad: {assortativity:.4f}")

Número de nodos: 143
Número de aristas: 623
Grado promedio: 8.71
Coeficiente de clustering promedio: 0.4339
Longitud de camino más corto promedio: 2.9670

Comparación con red aleatoria equivalente:
Coeficiente de clustering (aleatorio): 0.0535
Longitud de camino más corto (aleatorio): 2.5921

Coeficiente de asortatividad: -0.0195
